# Purpose of operating systems

At fundamental level we have applications and hardware, and the task of the OS is to translate between them. 




      - structure of simple computer, with processor, memory, storage and communication devices
      - forward reference to concurrency
      - discuss core abstractions of each and fundamental properties slow/fast

- translate physical abstractions to virtual/clean ones                                                              
- allow many applications to share resources                                                                         
- space and time multiplexing                                                                                        
- protect users from each other                                                                                      
- support many different types of computers                                                                          
- protect against attacks                                                                                            
- enable control/configuration                                                                                       
- enable devices                                                                                                     
- summarize tasks of the operating system

# Why do we care
- core platform everything else relies on, toughest code to write
- means that everything relies on it

- is the fun over... talk about future/changes
